In [0]:
import os
import subprocess

In [0]:
dbutils.widgets.text("username", "", "Please enter your git username")
dbutils.widgets.text("password", "", "Please enter your password")

In [0]:
username = dbutils.widgets.get("username")
password = dbutils.widgets.get("password")
url = "https://" + username + ":" + password + "@github.com/MaximeRedstone/graphsMapReduce.git"
email = username + username + "@gmail.com"
dbutils.widgets.removeAll()

In [0]:
!git config --global user.email $email
!git config --global user.name $username
  
if os.path.exists(os.path.join(os.getcwd(), "graphsMapReduce")):
  !cd graphsMapReduce && git pull
else:
  !git clone $url

!cd graphsMapReduce/generator && make re
!mkdir graphsMapReduce/data

Cloning into 'graphsMapReduce'...
remote: Enumerating objects: 540, done.
remote: Counting objects: 0% (1/540)remote: Counting objects: 1% (6/540)remote: Counting objects: 2% (11/540)remote: Counting objects: 3% (17/540)remote: Counting objects: 4% (22/540)remote: Counting objects: 5% (27/540)remote: Counting objects: 6% (33/540)remote: Counting objects: 7% (38/540)remote: Counting objects: 8% (44/540)remote: Counting objects: 9% (49/540)remote: Counting objects: 10% (54/540)remote: Counting objects: 11% (60/540)remote: Counting objects: 12% (65/540)remote: Counting objects: 13% (71/540)remote: Counting objects: 14% (76/540)remote: Counting objects: 15% (81/540)remote: Counting objects: 16% (87/540)remote: Counting objects: 17% (92/540)remote: Counting objects: 18% (98/540)remote: Counting objects: 19% (103/540)remote: Counting objects: 20% (108/540)remote: Counting objects: 21% (114/540)remote: Counting objects: 22% (119/540)remote: Counting objects: 23% (125/540)remote: Counting objects: 24% (130/540)remote: Counting objects: 25% (135/540)remote: Counting objects: 26% (141/540)remote: Counting objects: 27% (146/540)remote: Counting objects: 28% (152/540)remote: Counting objects: 29% (157/540)remote: Counting objects: 30% (162/540)remote: Counting objects: 31% (168/540)remote: Counting objects: 32% (173/540)remote: Counting objects: 33% (179/540)remote: Counting objects: 34% (184/540)remote: Counting objects: 35% (189/540)remote: Counting objects: 36% (195/540)remote: Counting objects: 37% (200/540)remote: Counting objects: 38% (206/540)remote: Counting objects: 39% (211/540)remote: Counting objects: 40% (216/540)remote: Counting objects: 41% (222/540)remote: Counting objects: 42% (227/540)remote: Counting objects: 43% (233/540)remote: Counting objects: 44% (238/540)remote: Counting objects: 45% (243/540)remote: Counting objects: 46% (249/540)remote: Counting objects: 47% (254/540)remote: Counting objects: 48% (260/540)remote: Counting objects: 49% (265/540)remote: Counting objects: 50% (270/540)remote: Counting objects: 51% (276/540)remote: Counting objects: 52% (281/540)remote: Counting objects: 53% (287/540)remote: Counting objects: 54% (292/540)remote: Counting objects: 55% (297/540)remote: Counting objects: 56% (303/540)remote: Counting objects: 57% (308/540)remote: Counting objects: 58% (314/540)remote: Counting objects: 59% (319/540)remote: Counting objects: 60% (324/540)remote: Counting objects: 61% (330/540)remote: Counting objects: 62% (335/540)remote: Counting objects: 63% (341/540)remote: Counting objects: 64% (346/540)remote: Counting objects: 65% (351/540)remote: Counting objects: 66% (357/540)remote: Counting objects: 67% (362/540)remote: Counting objects: 68% (368/540)remote: Counting objects: 69% (373/540)remote: Counting objects: 70% (378/540)remote: Counting objects: 71% (384/540)remote: Counting objects: 72% (389/540)remote: Counting objects: 73% (395/540)remote: Counting objects: 74% (400/540)remote: Counting objects: 75% (405/540)remote: Counting objects: 76% (411/540)remote: Counting objects: 77% (416/540)remote: Counting objects: 78% (422/540)remote: Counting objects: 79% (427/540)remote: Counting objects: 80% (432/540)remote: Counting objects: 81% (438/540)remote: Counting objects: 82% (443/540)remote: Counting objects: 83% (449/540)remote: Counting objects: 84% (454/540)remote: Counting objects: 85% (459/540)remote: Counting objects: 86% (465/540)remote: Counting objects: 87% (470/540)remote: Counting objects: 88% (476/540)remote: Counting objects: 89% (481/540)remote: Counting objects: 90% (486/540)remote: Counting objects: 91% (492/540)remote: Counting objects: 92% (497/540)remote: Counting objects: 93% (503/540)remote: Counting objects: 94% (508/540)remote: Counting objects: 95% (513/540)remote: Counting objects: 96% (519/540)remote: Counting objects: 97% (524/540)remote: Counting objects: 98% (530/

In [0]:
def processGraph(filepath):
  
  def reduce_ccf(x):
    key = x[0]
    values = x[1]
    min_value = values.pop(values.index(min(values)))
    ret = []
    if min_value < key:
      ret.append((key, min_value))
      for value in values:
        accum.add(1) # là on peu surement add en une fois la taille de la liste
        ret.append((value, min_value)) # là on peut surement zip 
    return (ret)
  
  
  text_file = sc.textFile(filepath)
  
  text_file = text_file.filter(lambda x: "#" not in x)
  text_file_split = text_file.map(lambda x: x.split())
  input = text_file_split.map(lambda x: (int(x[0]), int(x[1])))

  accum = sc.accumulator(1)
  while accum.value > 0:

    accum.value = 0
    print("Start loop ", accum.value)

    # CCF-Iterate
    it_map = input.flatMap(lambda x: ((x[0], x[1]), (x[1], x[0])))
#     it_groupby = it_map.groupByKey().mapValues(lambda x: sorted(x))
    it_groupby = it_map.groupByKey().mapValues(list)
    it_reduce = it_groupby.flatMap(lambda x: reduce_ccf(x))
    
    # CCF-Dedup
    ded_map = it_reduce.map(lambda x: ((x[0], x[1]), None))
    ded_groupby = ded_map.groupByKey().mapValues(list)
    input = ded_groupby.map(lambda x: (x[0][0], x[0][1]))

    viz = input.collect()
    print("End loop ", accum.value)

  print("Processed file")
#   print(viz)

In [0]:
!ls graphsMapReduce/data

In [0]:
user_inputs = {
  "4M": 5,
  "7M": 5
}

for k, v in user_inputs.items():
  
  filename = "graph_" + k + ".txt"
  rel_filepath = os.path.join("..", "data", filename)
  abs_filepath = os.path.join("graphsMapReduce", "data",filename)
  !cd graphsMapReduce/generator && ./graph_generator $k $rel_filepath $v
  
  filestore_name = "dbfs:/FileStore/" + filename
  dbutils.fs.rm(filestore_name)
  f = open(abs_filepath, "r")
  string = f.read()
  dbutils.fs.put(filestore_name, string)
  
  processGraph(filestore_name)

Start program at Sat Feb 27 09:43:28 2021
End program at Sat Feb 27 09:43:28 2021
Wrote 4194476 bytes.
Start loop 0
End loop 587752
Start loop 0
End loop 1163317
Start loop 0
End loop 2276279
Start loop 0
End loop 1639836
Start loop 0
End loop 36684
Start loop 0
End loop 0
Processed file
Start program at Sat Feb 27 09:45:15 2021
End program at Sat Feb 27 09:45:16 2021
Wrote 7340198 bytes.
Start loop 0
End loop 990461
Start loop 0
End loop 1959321
Start loop 0
End loop 3869491
Start loop 0
End loop 3368440
Start loop 0
End loop 84002
Start loop 0
End loop 0
Processed file

In [0]:
event_dir = str(subprocess.check_output(['ls', 'eventlogs']))
event_dir = int(event_dir.split("\'")[1][:-2])
full_old_path = os.path.join('eventlogs', str(event_dir), 'eventlog')
full_new_path = os.path.join('eventlogs', str(event_dir), 'eventlog.json')

In [0]:
!cd graphsMapReduce && git pull

Already up to date.

In [0]:
comment = ""
!pip install logzero
!mv $full_old_path $full_new_path
!cd graphsMapReduce/src && python3 main.py -e $event_dir #-c $comment

[I 210227 09:49:17 spark_logger:212] Starting generate_jobs
[I 210227 09:49:17 spark_logger:215] Starting unpack_stages_from_jobs
[I 210227 09:49:17 spark_logger:218] Starting generate_stages
[I 210227 09:49:17 spark_logger:221] Starting generate_tasks
[I 210227 09:49:18 spark_logger:157] Writing logs for files ['graph_4M.txt', 'graph_7M.txt']

In [0]:
!cd graphsMapReduce/generator && make fclean
!cd graphsMapReduce && git add -A
!cd graphsMapReduce && git config --global user.email "sparkistansparkistan@gmail.com"
!cd graphsMapReduce && git config --global user.name "sparkistan"
!cd graphsMapReduce && git commit -m "Added logs"
!cd graphsMapReduce && git push $url

make clean graph_generator
rm -f .o files
make fclean graph_generator
rm -f .o files
rm -f libft.a
rm -f ft_printf
[main d3f4b70] Added logs
 10 files changed, 1734 insertions(+), 2655 deletions(-)
 rewrite cleaned_logs/graph_4M.txt/log_accumulables.csv (96%)
 rewrite cleaned_logs/graph_4M.txt/log_jobs.csv (83%)
 rewrite cleaned_logs/graph_4M.txt/log_rdds.csv (86%)
 rewrite cleaned_logs/graph_4M.txt/log_stages.csv (60%)
 rewrite cleaned_logs/graph_4M.txt/log_tasks.csv (98%)
 rewrite cleaned_logs/graph_7M.txt/log_accumulables.csv (96%)
 rewrite cleaned_logs/graph_7M.txt/log_jobs.csv (70%)
 rewrite cleaned_logs/graph_7M.txt/log_rdds.csv (87%)
 rewrite cleaned_logs/graph_7M.txt/log_tasks.csv (96%)
Counting objects: 15, done.
Delta compression using up to 2 threads.
Compressing objects: 6% (1/15) Compressing objects: 13% (2/15) Compressing objects: 20% (3/15) Compressing objects: 26% (4/15) Compressing objects: 33% (5/15) Compressing objects: 40% (6/15) Compressing objects: 46% (7/15) Compressing objects: 53% (8/15) Compressing objects: 60% (9/15) Compressing objects: 66% (10/15) Compressing objects: 73% (11/15) Compressing objects: 80% (12/15) Compressing objects: 86% (13/15) Compressing objects: 93% (14/15) Compressing objects: 100% (15/15) Compressing objects: 100% (15/15), done.
Writing objects: 6% (1/15) Writing objects: 13% (2/15) Writing objects: 20% (3/15) Writing objects: 26% (4/15) Writing objects: 33% (5/15) Writing objects: 40% (6/15) Writing objects: 46% (7/15) Writing objects: 53% (8/15) Writing objects: 66% (10/15) Writing objects: 73% (11/15) Writing objects: 80% (12/15) Writing objects: 86% (13/15) Writing objects: 93% (14/15) Writing objects: 100% (15/15) Writing objects: 100% (15/15), 13.51 KiB | 2.25 MiB/s, done.
Total 15 (delta 11), reused 0 (delta 0)
remote: Resolving deltas: 0% (0/11)remote: Resolving deltas: 9% (1/11)remote: Resolving deltas: 18% (2/11)remote: Resolving deltas: 27% (3/11)remote: Resolving deltas: 36% (4/11)remote: Resolving deltas: 45% (5/11)remote: Resolving deltas: 54% (6/11)remote: Resolving deltas: 63% (7/11)remote: Resolving deltas: 72% (8/11)remote: Resolving deltas: 81% (9/11)remote: Resolving deltas: 90% (10/11)remote: Resolving deltas: 100% (11/11)remote: Resolving deltas: 100% (11/11), completed with 6 local objects.
To https://github.com/MaximeRedstone/graphsMapReduce.git
 0df9dc3..d3f4b70 main -> main